In [1]:
import pandas as pd
import requests
import logging
import time
import re
import string

# Params

In [2]:
# Set your Google API key here. 
# Even if using the free 2500 queries a day, its worth getting an API key since the rate limit is 50 / second.
# With API_KEY = None, you will run into a 2 second delay every 10 requests or so.
# With a "Google Maps Geocoding API" key from https://console.developers.google.com/apis/, 
# the daily limit will be 2500, but at a much faster rate.
with open('../../data/keys/google/API_KEY','r') as f:
    API_KEY=eval(f.readline())
    
# Backoff time sets how many minutes to wait between google pings when your API limit is hit
BACKOFF_TIME = 30

# Return Full Google Results? If True, full JSON results from Google are included in output
RETURN_FULL_RESULTS = True

# Include country bias to increase accuracy
BIAS = False

In [3]:
logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)

# create console handler
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)

# Load Data

In [4]:
path_to_data = '../../data/locations/profiles/geocoding/'
input_file   = 'account-locations-to-geocode-manually.pkl'
output_file  = 'account-locations-geocoded-manually.pkl'

In [5]:
locations = pd.read_pickle(path_to_data+input_file)
print('# Locations:',locations.shape[0])

# Locations: 659


In [6]:
locations.head()

,LOCATION,FIXED
0,Liverpool,"Liverpool, UK"
1,Nederland,Netherland
2,Cali,"Baja California, Mexico"
3,PH,Philippines
4,liverpool,"Liverpool, UK"


In [7]:
# Include bias
def include_bias(x):
    if ', ' + x['BIAS'] not in x['LOCATION']:
        return x['LOCATION'] + ', ' + x['BIAS']
    else:
        return x['LOCATION']

if BIAS:
    print('Bias included')
    # Make a big list of all of the addresses to be processed.
    locations['ADDRESS'] = locations.apply(include_bias, 1).tolist()
else:
    print('Bias not included')
    if 'FIXED' not in locations.columns:
        locations['ADDRESS'] = locations['LOCATION'].tolist()
    else:
        locations['ADDRESS'] = locations['FIXED'].tolist()

Bias not included


In [8]:
print('First 10 Requests:\n\n', '\n'.join(locations['ADDRESS'][:10]))

First 10 Requests:

 Liverpool, UK
Netherland
Baja California, Mexico
Philippines
Liverpool, UK
Kent, UK
Florida, USA
USA
Portsmouth, UK
Baja California, Mexico


# Google Geocoding

Python script for batch geocoding of addresses using the Google Geocoding API.

This script allows for massive lists of addresses to be geocoded for free by pausing when the 
geocoder hits the free rate limit set by Google (2500 per day).  If you have an API key for paid
geocoding from Google, set it in the API key section.

Addresses for geocoding can be specified in a list of strings "addresses". In this script, addresses
come from a csv file with a column "Address". Adjust the code to your own requirements as needed.
After every 100 successul geocode operations, a temporary file with results is recorded in case of 
script failure / loss of connection later.

Addresses and data are held in memory, so this script may need to be adjusted to process files line
by line if you are processing millions of entries.

In [9]:
def get_google_results(address, api_key=None, return_full_response=False):
    """
    Get geocode results from Google Maps Geocoding API.
    
    Note, that in the case of multiple google geocode reuslts, this function returns details of the FIRST result.
    
    @param address: String address as accurate as possible. For Example "18 Grafton Street, Dublin, Ireland"
    @param api_key: String API key if present from google. 
                    If supplied, requests will use your allowance from the Google API. If not, you
                    will be limited to the free usage of 2500 requests per day.
    @param return_full_response: Boolean to indicate if you'd like to return the full response from google. This
                    is useful if you'd like additional location details for storage or parsing later.
    """
    # Set up your Geocoding url
    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(address)
    if api_key is not None:
        geocode_url = geocode_url + "&key={}".format(api_key)
        
    # Ping google for the reuslts:
    results = requests.get(geocode_url)
    # Results will be in JSON format - convert to dict using requests functionality
    results = results.json()
    
    # if there's no results or an error, return empty results.
    if len(results['results']) == 0:
        output = {
            "formatted_address" : None,
            "latitude": None,
            "longitude": None,
            "accuracy": None,
            "google_place_id": None,
            "type": None,
            "postcode": None
        }
    else:    
        answer = results['results'][0]
        output = {
            "formatted_address" : answer.get('formatted_address'),
            "latitude": answer.get('geometry').get('location').get('lat'),
            "longitude": answer.get('geometry').get('location').get('lng'),
            "accuracy": answer.get('geometry').get('location_type'),
            "google_place_id": answer.get("place_id"),
            "type": ",".join(answer.get('types')),
            "postcode": ",".join([x['long_name'] for x in answer.get('address_components') 
                                  if 'postal_code' in x.get('types')])
        }
        
    # Append some other details:    
    output['input_string'] = address
    output['number_of_results'] = len(results['results'])
    output['status'] = results.get('status')
    if return_full_response is True:
        output['response'] = results
    
    return output

In [10]:
# Ensure, before we start, that the API key is ok/valid, and internet access is ok
test_result = get_google_results("London, England", API_KEY, RETURN_FULL_RESULTS)
if (test_result['status'] != 'OK') or (test_result['formatted_address'] != 'London, UK'):
    logger.warning("There was an error when testing the Google Geocoder.")
    raise ConnectionError('Problem with test results from Google Geocode - check your API key and internet connection.')

In [11]:
# Create a list to hold results
results = []

# Go through each address in turn
for idx,location in locations.iterrows():
    
    # While the address geocoding is not finished:
    geocoded = False
    
    while geocoded is not True:
        
        # Geocode the address with google
        try:
            
            geocode_result = get_google_results(location['ADDRESS'], 
                                                API_KEY, 
                                                return_full_response=RETURN_FULL_RESULTS)
            
            geocode_result['raw_string'] = location['LOCATION']
            
        except Exception as e:
            
            logger.exception(e)
            
            logger.error("Major error with {}".format(location['ADDRESS']))
            
            logger.error("Skipping!")
            
            geocoded = True
            
        # If we're over the API limit, backoff for a while and try again later.
        if geocode_result['status'] == 'OVER_QUERY_LIMIT':
            
            logger.info("Hit Query Limit! Backing off for a bit.")
            
            time.sleep(BACKOFF_TIME * 60) # sleep for 30 minutes
            
            geocoded = False
            
        else:
            
            # If we're ok with API use, save the results
            # Note that the results might be empty / non-ok - log this
            if geocode_result['status'] != 'OK':
                
                logger.warning("Error geocoding {}: {}".format(location['ADDRESS'], geocode_result['status']))
                
            logger.debug("Geocoded: {}: {}".format(location['ADDRESS'], geocode_result['status']))
            
            results.append(geocode_result)           
            
            geocoded = True

    # Print status every 100 addresses
    if len(results) % 100 == 0:
        
        logger.info("Completed {} of {} address".format(len(results), len(locations)))
            
    # Every 100 addresses, save progress to file(in case of a failure so you have something!)
    if len(results) % 1000 == 0:
        
        pd.DataFrame(results).to_pickle("{}_bak".format(path_to_data+output_file))

# All done
logger.info("Finished geocoding all addresses")

Geocoded: Liverpool, UK: OK
Geocoded: Netherland: OK
Geocoded: Baja California, Mexico: OK
Geocoded: Philippines: OK
Geocoded: Liverpool, UK: OK
Geocoded: Kent, UK: OK
Geocoded: Florida, USA: OK
Geocoded: USA: OK
Geocoded: Portsmouth, UK: OK
Geocoded: Baja California, Mexico: OK
Geocoded: Netherland: OK
Geocoded: Lincoln, UK: OK
Geocoded: Yorkshire, UK: OK
Geocoded: Toledo, Spain: OK
Geocoded: Santander, Spain: OK
Geocoded: Durham, UK: OK
Geocoded: Austin, Texas: OK
Geocoded: Curitiba, Brazil: OK
Geocoded: New Jersey, USA: OK
Geocoded: Liverpool, UK: OK
Geocoded: Somerset, UK: OK
Geocoded: Kent, UK: OK
Geocoded: Baja California, Mexico: OK
Geocoded: Antwerpen, Netherlands: OK
Geocoded: Philippines: OK
Geocoded: Miami, USA: OK
Geocoded: Curitiba, Brazil: OK
Geocoded: Japan: OK
Geocoded: Dallas, Texas: OK
Geocoded: Panama city, Panama: OK
Geocoded: Portsmouth, UK: OK
Geocoded: Curitiba, Brazil: OK
Geocoded: Santiago, Chile: OK
Geocoded: Borneo: OK
Geocoded: Stafford, UK: OK
Geocoded: Phi

Geocoded: UK: OK
Geocoded: Indonesia: OK
Geocoded: Indonesia: OK
Geocoded: Xixón, ES: OK
Geocoded: Indonesia: OK
Geocoded: Canada: OK
Geocoded: Dundalk, Ireland: OK
Geocoded: Panama city, Panama: OK
Geocoded: Michigan, USA: OK
Geocoded: UK: OK
Geocoded: Indonesia: OK
Geocoded: Quito, Ecuador: OK
Geocoded: NY State, USA: OK
Geocoded: Michigan, USA: OK
Geocoded: Indonesia: OK
Geocoded: Guayaquil, Ecuador: OK
Geocoded: Turkey: OK
Geocoded: UK: OK
Geocoded: NY State, USA: OK
Geocoded: Indonesia: OK
Geocoded: Turkey: OK
Geocoded: Canada: OK
Geocoded: Indonesia: OK
Geocoded: Turkey: OK
Geocoded: Turkey: OK
Geocoded: Texas, USA: OK
Geocoded: Borneo: OK
Geocoded: Turkey: OK
Geocoded: Indonesia: OK
Geocoded: South Dakota: OK
Geocoded: Florida, USA: OK
Geocoded: Indonesia: OK
Geocoded: Texas, USA: OK
Geocoded: Indonesia: OK
Geocoded: Maine, USA: OK
Geocoded: Thailand: OK
Geocoded: Yogyakarta: OK
Geocoded: Indonesia: OK
Geocoded: Indonesia: OK
Geocoded: Turkey: OK
Error geocoding Bumi Serpong Dam

Geocoded: Singapore, Singapore: OK
Geocoded: Baixada Santista, Brazil: OK
Geocoded: Indonesia: OK
Geocoded: Zanzibar: OK
Geocoded: Somerset, UK: OK
Geocoded: UK: OK
Geocoded: Panama city, Panama: OK
Geocoded: Pittsburg, USA: OK
Geocoded: UK: OK
Geocoded: Borispil, Ukrain: OK
Geocoded: New York, USA: OK
Geocoded: Indonesia: OK
Geocoded: Indonesia: OK
Geocoded: Curitiba: OK
Geocoded: San Petersburg, Russia: OK
Geocoded: UK: OK
Geocoded: Indonesia: OK
Geocoded: Turkey: OK
Geocoded: Guayaquil, Ecuador: OK
Geocoded: Indonesia: OK
Geocoded: Dallas, Texas: OK
Geocoded: Indonesia: OK
Geocoded: Indonesia: OK
Geocoded: Abu Dhabi, United Arab Emirates: OK
Geocoded: Miami, USA: OK
Geocoded: Indonesia: OK
Geocoded: England, UK: OK
Geocoded: Panama city, Panama: OK
Geocoded: Turkey: OK
Geocoded: Bronx, NY, USA: OK
Geocoded: Indonesia: OK
Geocoded: Indonesia: OK
Geocoded: Manilla, Philippines: OK
Geocoded: UK: OK
Geocoded: Semarang, Indonesia: OK
Geocoded: Indonesia: OK
Geocoded: Florya, Istanbul, Tu

In [15]:
print('SAVE:')

# Write the full results to csv using the pandas library.
pd.DataFrame(results).to_pickle(path_to_data+output_file)

print('DONE!')

SAVE:
DONE!
